In [60]:
import time
import os

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains


In [61]:
def getPageSource(hotel_url):
    driver = webdriver.Firefox()
    driver.get(hotel_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    return driver, soup    

In [62]:
def getReviews(reviews, dict_):
    for review in reviews:
        review_head = review.find("a", class_="fCitC").get_text()
        review = review.find("q", class_="XllAv H4 _a").get_text()
        dict_[review_head] = review
        

In [63]:
def getPositiveReviews(hotel_url):
    positiveReviews = {}
    
    driver, soup = getPageSource(hotel_url)
    
#     driver.find_element_by_xpath(f".//label[contains(text(),'Very Good')]").click()
    driver.find_element_by_xpath(f".//label[contains(text(),'Excellent')]").click()
    
    reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
    
    while True:        
        getReviews(reviews, positiveReviews)
        
        pagination = soup.find("div", class_="ui_pagination is-centered")
        next_button = pagination.find("a", class_="ui_button nav next primary")
        
        if next_button:
            if "disabled" not in next_button['class']:
                driver.find_element_by_link_text("Next").click()
                time.sleep(3)       
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                reviews = soup.find_all("div", class_="cWwQK MC R2 Gi z Z BB dXjiy")
                continue
            else:
                break
        break
    
    driver.close()
    return positiveReviews


In [64]:
hotel_url = 'https://www.tripadvisor.com/Hotel_Review-g293890-d305977-Reviews-Hotel_Yak_Yeti-Kathmandu_Kathmandu_Valley_Bagmati_Zone_Central_Region.html'
positive = getPositiveReviews(hotel_url)

In [65]:
positiveDataframe = pd.DataFrame({'heading': positive.keys(),
                                  'reviews': positive.values()})

In [66]:
positiveDataframe

,heading,reviews
0,Historic and Hospitality unmatched,What a wonderful stay in Kathmandu!!! I was t...
1,The hotel has a lot to improve in terms of hyg...,Needs a lot of improvement. The water had a ve...
2,Hotel Quarantine @ Hotel Manaslu,"Warm hospitality, clean and colorful environme..."
3,Tranquillity amidst the mayhem,Enjoy traditional Nepalese hospitality in the ...
4,Nice hotel in a good location,Having just returned from an Everest Base Camp...
5,An authentic Nepali experience,"The hotel is extremely comfortable, right in t..."
6,Oh The Vibe and feel of the Hotel! Authentic &...,"The architecture, the place, the hospitality, ..."
7,A calm and tranquil oasis in Kathmandu,Feel calm and well cared for in this tradition...
8,An Oasis of peace,A quaint hotel decorated in local style. Room ...
9,"Beautiful hotel, so pleasant!","2nd year we stay at Manaslu pre and post trek,..."


In [67]:
def concatExcel():
    positiveExcel = pd.read_excel('positive.xlsx', header=0, index_col=0)
    positiveConcat = pd.concat([positiveExcel, positiveDataframe])
    positiveConcat.to_excel('positive.xlsx')

In [69]:
# concatExcel()